In [ ]:
pip install atproto


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.9/671.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.7 MB/s eta 0:00:00


In [ ]:
pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install tensorflow


In [ ]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time

# Inserindo o Bearer Token
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAPQSwwEAAAAARcHV%2F4skUW5DdDXjAL%2BONWPgIww%3DMBVgeRHKjjWvFt9mmagFb2YAOPMi0i3skCkeYYfSHUzzjJCvUk'

# Configurações de autenticação
def criar_headers():
    return {"Authorization": f"Bearer {BEARER_TOKEN}"}

# Inicializar o analisador de sentimentos VADER
analyzer = SentimentIntensityAnalyzer()

# Função para coletar tweets recentes de um usuário e analisar o sentimento
def coletar_tweets_e_analisar_sentimento(nome_usuario, max_requisicoes=3):
    try:
        headers = criar_headers()
        url = "https://api.twitter.com/2/tweets/search/recent"

        # Configuração da consulta
        query = f"from:{nome_usuario} -is:retweet"
        params = {"query": query, "max_results": 1, "tweet.fields": "created_at"}

        sentimentos_tweets = []
        requisicoes_realizadas = 0

        # Loop para controlar o número de requisições
        while requisicoes_realizadas < max_requisicoes:
            response = requests.get(url, headers=headers, params=params)

            # Verifica se o status é 429 (Too Many Requests)
            if response.status_code == 429:
                print("Erro 429: Limite de requisições excedido. Aguardando para tentar novamente.")
                time.sleep(900)  # Espera de 15 minutos antes de tentar novamente
                continue
            elif response.status_code != 200:
                print(f"Erro: {response.status_code}, {response.json()}")
                break

            tweets = response.json().get("data", [])
            print(f"Total de tweets coletados na requisição {requisicoes_realizadas + 1}: {len(tweets)}")

            # Analisar o sentimento de cada tweet coletado
            for tweet in tweets:
                conteudo = tweet["text"]
                print(f"Conteúdo do tweet: {conteudo}")

                sentimento = analyzer.polarity_scores(conteudo)
                sentimentos_tweets.append(sentimento)

            requisicoes_realizadas += 1
            time.sleep(60)  # Pausa de 1 minuto para evitar limite de taxa

        return sentimentos_tweets

    except Exception as e:
        print(f"Erro ao coletar tweets de {nome_usuario}: {str(e)}")
        return None

# Função para exibir os resultados
def exibir_resultados(sentimentos_tweets):
    if sentimentos_tweets:
        print("Sentimentos dos tweets:")
        for i, sentimento in enumerate(sentimentos_tweets):
            print(f"Tweet {i+1}: {sentimento}")

# Função principal
def main():
    nome_usuario_influenciador = '@maisa'  # Substitua pelo nome de usuário desejado
    sentimentos_tweets = coletar_tweets_e_analisar_sentimento(nome_usuario_influenciador)
    exibir_resultados(sentimentos_tweets)

# Executar a função principal
if __name__ == "__main__":
    main()


Erro 429: Limite de requisições excedido. Aguardando para tentar novamente.
Erro: 503, {'title': 'Service Unavailable', 'detail': 'Service Unavailable', 'type': 'about:blank', 'status': 503}
